## Primero generando los datos adversarios

In [8]:
import torch
import numpy as np
import pandas as pd
import random 
from Evaluator import ModelEvaluator

## cargando los datos adversarios

In [16]:
advData=pd.read_csv('datasets/Adversarial_data.csv')
print(advData.head(10))


   -97.0  -47.0   -6.0  -27.0  73.0  13.0  -59.0  93.0  -36.0   5.0
0   -1.0  -65.0  -26.0   51.0  -9.0  23.0   63.0 -54.0   19.0  15.0
1   -1.0   63.0  -93.0   43.0 -12.0  76.0   -4.0 -53.0   62.0  50.0
2   53.0  -25.0   42.0   87.0   1.0   1.0   18.0  -6.0   83.0  11.0
3  -79.0  -62.0  -21.0   83.0   0.0  46.0  -52.0  13.0  -43.0  21.0
4   22.0  -87.0   55.0  -50.0 -24.0  73.0  -10.0 -69.0   74.0  26.0
5    0.0   -5.0  -47.0  -15.0 -89.0  97.0  -97.0  91.0   79.0  40.0
6 -120.0  -12.0  -68.0   23.0 -37.0 -37.0  -82.0  12.0    1.0  76.0
7  -72.0   85.0 -101.0   16.0 -94.0  -3.0   59.0 -58.0   47.0  91.0
8  -67.0    7.0  -99.0  -84.0 -57.0 -15.0 -102.0  39.0  -38.0  37.0
9  -65.0  -57.0    9.0    6.0 -56.0  44.0  -59.0  51.0  -71.0  90.0


In [17]:
advData.columns=['tcp_packets','dist_port_tcp','external_ips','vulume_bytes','udp_packets','source_app_packets','remote_app_packets','source_app_bytes','remote_app_bytes','dns_query_times']

Agregando los headers, a partir del conocimiento de los datos originales, teniendo en cuenta que el entrenamiento no es un proceso que realiza el atacante.

In [18]:
advData.head(10)

,tcp_packets,dist_port_tcp,external_ips,vulume_bytes,udp_packets,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,dns_query_times
0,-1.0,-65.0,-26.0,51.0,-9.0,23.0,63.0,-54.0,19.0,15.0
1,-1.0,63.0,-93.0,43.0,-12.0,76.0,-4.0,-53.0,62.0,50.0
2,53.0,-25.0,42.0,87.0,1.0,1.0,18.0,-6.0,83.0,11.0
3,-79.0,-62.0,-21.0,83.0,0.0,46.0,-52.0,13.0,-43.0,21.0
4,22.0,-87.0,55.0,-50.0,-24.0,73.0,-10.0,-69.0,74.0,26.0
5,0.0,-5.0,-47.0,-15.0,-89.0,97.0,-97.0,91.0,79.0,40.0
6,-120.0,-12.0,-68.0,23.0,-37.0,-37.0,-82.0,12.0,1.0,76.0
7,-72.0,85.0,-101.0,16.0,-94.0,-3.0,59.0,-58.0,47.0,91.0
8,-67.0,7.0,-99.0,-84.0,-57.0,-15.0,-102.0,39.0,-38.0,37.0
9,-65.0,-57.0,9.0,6.0,-56.0,44.0,-59.0,51.0,-71.0,90.0


In [20]:
advData['type']='malicious'

In [21]:
advData.head(5)

,tcp_packets,dist_port_tcp,external_ips,vulume_bytes,udp_packets,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,dns_query_times,type
0,-1.0,-65.0,-26.0,51.0,-9.0,23.0,63.0,-54.0,19.0,15.0,malicious
1,-1.0,63.0,-93.0,43.0,-12.0,76.0,-4.0,-53.0,62.0,50.0,malicious
2,53.0,-25.0,42.0,87.0,1.0,1.0,18.0,-6.0,83.0,11.0,malicious
3,-79.0,-62.0,-21.0,83.0,0.0,46.0,-52.0,13.0,-43.0,21.0,malicious
4,22.0,-87.0,55.0,-50.0,-24.0,73.0,-10.0,-69.0,74.0,26.0,malicious


## load discriminator

In [10]:
eval=ModelEvaluator()
eval.test(AdvData[:1])

array(['benign'], dtype=object)

## Secure learning